In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
titanic_file_path = '../input/titanic/train.csv'
titanic_data = pd.read_csv(titanic_file_path) 
titanic_data.describe()

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [2]:
titanic_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [3]:
titanic_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df = titanic_data.filter(items=['Survived','Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'])
y = df.Survived
X = df.drop(['Survived'], axis=1)

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# imputation
X_train["Embarked"].fillna('S', inplace =True)
X_train['Age'].fillna(X_train['Age'].mean(), inplace = True)

X_valid["Embarked"].fillna('S', inplace =True)
X_valid['Age'].fillna(X_train['Age'].mean(), inplace = True)

In [5]:
X_train.corr()

,Pclass,Age,SibSp,Parch,Fare
Pclass,1.000000,-0.333687,0.093648,0.017336,-0.534776
Age,-0.333687,1.000000,-0.220956,-0.169844,0.096597
SibSp,0.093648,-0.220956,1.000000,0.423676,0.145538
Parch,0.017336,-0.169844,0.423676,1.000000,0.213608
Fare,-0.534776,0.096597,0.145538,0.213608,1.000000


In [6]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[['Sex']]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[['Sex']]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop('Sex', axis=1)
num_X_valid = X_valid.drop('Sex', axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

X_train = OH_X_train.rename(columns={0:'Female', 1:'Male'})
X_valid = OH_X_valid.rename(columns={0:'Female', 1:'Male'})

# OH for 'Embarked'
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[['Embarked']]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[['Embarked']]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop('Embarked', axis=1)
num_X_valid = X_valid.drop('Embarked', axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

X_train = OH_X_train.rename(columns={0:'C', 1:'Q', 2:'S'})
X_valid = OH_X_valid.rename(columns={0:'C', 1:'Q', 2:'S'})

In [7]:
from xgboost import XGBRegressor

my_model = XGBRegressor(n_estimators=500, learning_rate=0.05, early_stopping_rounds=5)
my_model.fit(X_train, y_train,
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=5, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.05, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=500, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

# Test set predictions

In [8]:
test_data_path = '../input/titanic/test.csv'

# read test data file using pandas
test_data = pd.read_csv(test_data_path)

X_test = test_data.filter(items=['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'])

X_test["Embarked"].fillna('S', inplace =True)
X_test['Age'].fillna(X_train['Age'].mean(), inplace = True)

In [9]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_test = pd.DataFrame(OH_encoder.fit_transform(X_test[['Sex']]))

# One-hot encoding removed index; put it back
OH_cols_test.index = X_test.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_test = X_test.drop('Sex', axis=1)

# Add one-hot encoded columns to numerical features
OH_X_test = pd.concat([num_X_test, OH_cols_test], axis=1)

X_test = OH_X_test.rename(columns={0:'Female', 1:'Male'})

In [10]:
# OH for 'Embarked'
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_test = pd.DataFrame(OH_encoder.fit_transform(X_test[['Embarked']]))

# One-hot encoding removed index; put it back
OH_cols_test.index = X_test.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_test = X_test.drop('Embarked', axis=1)

# Add one-hot encoded columns to numerical features
OH_X_test = pd.concat([num_X_test, OH_cols_test], axis=1)

X_test = OH_X_test.rename(columns={0:'C', 1:'Q', 2:'S'})

In [11]:
from sklearn.metrics import accuracy_score
xgb_predictions = my_model.predict(X_test)

t=[]
for i in xgb_predictions:
    if i <= 0.5:
        t.append(0)
    else:
        t.append(1)

In [12]:
y_pred = pd.DataFrame(t)
y_pred.rename(columns={0:'Survived'})
y_pred['PassengerId'] = test_data['PassengerId']

In [13]:
# Run the code to save predictions in the format used for competition scoring
y_pred.to_csv('submission.csv', index=False)